In [ ]:
# Stdlib imports
from contextlib import ExitStack, closing, contextmanager
from itertools import count
from enum import Enum
import socket

# Third-party imports
import msgpack

# Local imports


In [ ]:
# Globals
BINDADDR = '127.0.0.1'
BINDPORT = 9999
BUFFER_SIZE = 1024

In [ ]:
# Protocol enums
class MessageType(Enum):
    """MessageType"""
    #  A message initiating a request.
    Request = 0

    #  A message sent in response to a request.
    Response = 1

    #  A message notifying of some additional information.
    Notification = 2


class SessionType(Enum):
    """SessionType"""

    # Session used when an agent is starting.
    #
    # Indicates that authentication services are likely to not be available.
    # Only the GetKeyFile request is available within a bootstrap session.
    Boot = 0

    # Authentication is used to allow access to keyfiles.
    #
    # All request types are available within an authenticated session.
    Auth = 1


class AuthMessage(Enum):
    """AuthMessage"""

    # Retrieve the keyfile.
    #
    # Requires 1 argument: key. Only succeeds if the keyfile exists.
    GetKeyFile = 0

    # Create the keyfile.
    #
    # Requires 2 arguments: key, keyfile. Only succeeds if the keyfile does
    # not exist.
    CreateKeyFile = 1

    # Change only the keyfile
    #
    # Requires 2 arguments: key, new keyfile. Only succeeds if the keyfile
    # already exists.
    ChangeKeyFile = 2

    # Change only the key
    #
    # Requires 2 arguments: old key, new key. Only succeeds if the keyfile
    # already exists.
    ChangeKey = 3

    # Replace the keyfile
    #
    # Requires 3 arguments: Old key, new key, new keyfile. Only succeeds if
    # the keyfile already exists.
    ReplaceKeyFile = 4

    # Delete the keyfile.
    #
    # requires 1 argument: key. Only succeeds if the keyfile already exists.
    DeleteKeyFile = 5

    # Check if a key exists
    #
    # requires 1 argument: key. Always succeeds and returnes true or false.
    KeyExists = 6


class AuthError(Enum):
    """AuthError"""
    Nil = 0

    # Key file is not found.
    KeyFileNotFound = 1

    # Key file exists.
    KeyFileExists = 2

    # DB error
    DatabaseError = 3


# Used with the notification rpc message type.
class AuthNotice(Enum):
    """AuthNotice"""
    # No more requests will be made
    Done = 2



In [ ]:
# General RPC Messages
def start_session():
    """start session"""
    msgtype = MessageType.Notification
    code = SessionType.Auth
    args = []
    msg = [msgtype.value, code.value, args]
    return msgpack.packb(msg)


def request(msgid, *args, method=None):
    """request"""
    if not isinstance(msgid, int):
        msg = 'msgid arg expected int, got {}'
        raise TypeError(msg.format(msgid.__class__.__name__))
    if not isinstance(method, AuthMessage):
        msg = 'method arg expected AuthMessage, got {}'
        raise TypeError(msg.format(type(method).__name__))

    msgtype = MessageType.Request
    args = list(args)

    # Create message
    msg = [msgtype.value, msgid, method.value, args]
    return msgpack.packb(msg)


def notify(*args, code=None):
    """notify"""
    if not isinstance(code, AuthNotice):
        msg = 'method arg expected AuthNotice, got {}'
        raise TypeError(msg.format(type(code).__name__))
    msgtype = MessageType.Notification
    args = list(args)

    # Create message
    msg = [msgtype.value, code.value, args]
    return msgpack.packb(msg)


def send_message(conn, msg):
    # Send message
    conn.send(msg)
    
    # Get response
    data = conn.recv(BUFFER_SIZE)

    # Unpack
    return msgpack.unpackb(data)


class Response:
    def __init__(self, msg):
        self._msg = msg
        
    @property
    def id(self):
        return self._msg[1]
        
    @property
    def error(self):
        return AuthError(self._msg[2])
        
    @property
    def result(self):
        return self._msg[3]

In [ ]:
# RPC messages

def request_keyexists(conn, msgid, key):
    """request_keyexists"""
    if not isinstance(key, str):
        errmsg = ('key arg expected str, got {} '
                  'instead').format(type(key).__name__)
        raise TypeError(errmsg)

    key = key.encode('utf-8')
    msg = request(msgid, key, method=AuthMessage.KeyExists)
    response = send_message(conn, msg)
    return Response(response)


def request_getkeyfile(conn, msgid, key):
    """GetKeyFile"""
    if not isinstance(key, str):
        errmsg = ('key arg expected str, got {} '
                  'instead').format(type(key).__name__)
        raise TypeError(errmsg)

    key = key.encode('utf-8')
    msg = request(msgid, key, method=AuthMessage.GetKeyFile)
    response = send_message(conn, msg)
    return Response(response)


def request_createkeyfile(conn, msgid, key, keyfile):
    """request_createkeyfile"""
    values = [('key', key), ('keyfile', keyfile)]
    for name, val in values:
        if not isinstance(val, str):
            errmsg = ('{} arg expected str, got {} '
                      'instead').format(name, type(val).__name__)
            raise TypeError(errmsg)

    key = key.encode('utf-8')
    keyfile = keyfile.encode('utf-8')

    msg = request(msgid, key, keyfile, method=AuthMessage.CreateKeyFile)
    response = send_message(conn, msg)
    return Response(response)


def request_deletekeyfile(conn, msgid, key):
    """request_deletekeyfile"""
    if not isinstance(key, str):
        errmsg = ('{} arg expected str, got {} '
                  'instead').format(name, type(val).__name__)
        raise TypeError(errmsg)

    key = key.encode('utf-8')
    msg = request(msgid, key, method=AuthMessage.DeleteKeyFile)
    response = send_message(conn, msg)
    return Response(response)

    
def notify_done():
    """notify_done"""
    return notify(code=AuthNotice.Done)


In [ ]:
@contextmanager
def connection():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    with closing(sock) as s:
        s.connect((BINDADDR, BINDPORT))
        yield s
      
    
@contextmanager
def session(conn):
    # Start session
    conn.send(start_session())
    try:
        yield
    finally:
        # Finish session
        conn.send(notify_done())

In [ ]:
# Client Session
with ExitStack() as stack:
    # Setup connection and session
    conn = stack.enter_context(connection())
    stack.enter_context(session(conn))
    msgid = count()
    
    # Check if file exists, create it, and then get the file
    response = request_keyexists(conn, next(msgid), '42')
    if response.result:
        response = request_getkeyfile(conn, next(msgid), '42')
    else:
        request_createkeyfile(conn, next(msgid), '42', 'The Answer to Life, the Universe, and Everything')
        response = request_getkeyfile(conn, next(msgid), '42')
        
    answer = response.result.decode('utf-8')

    # Cleanup
    response = request_deletekeyfile(conn, next(msgid), '42')
    assert response.result
    
print('What is 42?')
print(answer)